In [110]:
#Import Dependencies
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

print(tf.__version__)



2.3.0


In [249]:

chosenstats =['dr_2RecTds','dr_2TotalTds','dr_1Plays','careerRec','dr_2Plays','careerTotalTds','careerTotalYds','dr_1Rec','careerRecTds','careerRecYds',
'dr_2TotalYds','dr_2RecYds','dr_1RecTds','dr_1RecYds','dr_2Rec','dr_1TotalYds','dr_1TotalTds','ExcludeZeroFPTS']
    
# chosenstats = ['Conf','Class','Age', 'Wt', 'Forty', 'Vertical', 'BenchReps', 'BroadJump', 'ThreeCone', 'Shuttle',
# 'careerRec', 'careerRecYds', 'careerYPC', 'careerRecTds',
# 'careerRushAtt', 'careerRushYds', 'careerYPA', 'careerRushTds',
# 'careerPlays', 'careerTotalYds', 'careerYPT', 'careerTotalTds',
# 'dr_1games', 'dr_1Rec', 'dr_1RecYds',
# 'dr_1YPC', 'dr_1RecTds', 'dr_1RushAtt', 'dr_1RushYds', 'dr_1YPA',
# 'dr_1RushTds', 'dr_1Plays', 'dr_1TotalYds', 'dr_1YPT', 'dr_1TotalTds',
# 'dr_2games', 'dr_2Rec', 'dr_2RecYds', 'dr_2YPC', 'dr_2RecTds',
# 'dr_2RushAtt', 'dr_2RushYds', 'dr_2YPA', 'dr_2RushTds', 'dr_2Plays',
# 'dr_2TotalYds', 'dr_2YPT', 'dr_2TotalTds', 'dr_3games', 'dr_3Rec',
# 'dr_3RecYds', 'dr_3YPC', 'dr_3RecTds', 'dr_3RushAtt', 'dr_3RushYds',
# 'dr_3YPA', 'dr_3RushTds', 'dr_3Plays', 'dr_3TotalYds', 'dr_3YPT',
# 'dr_3TotalTds','ExcludeZeroFPTS']

#read in the model csv
model_df = pd.read_csv("modelPython.csv")

predict2020 = pd.read_csv('draft2020.csv')
#create a df of the subjects matched by ID
subjects = model_df[['ID','Player_x','FirstNm','LastNm']]
subjects2020 = predict2020[['ID','Player_x','FirstNm','LastNm']]

# Convert Age, weight, and height columns to floats
model_df[['Wt', 'height']] = model_df[['Wt', 'height']].astype(float)

model_df.drop(columns = ['Player_x','FirstNm','LastNm'], inplace = True)

#Add year and ID to chosenstats
chosenstats.append('year')
chosenstats.append('ID')
chosenstats.append('FPTS')

if 'ExcludeZeroFPTS' in chosenstats:
    exclude = True
    chosenstats.remove('ExcludeZeroFPTS')
else:
    pass

model_df=model_df[chosenstats]


#drop 1 columns associated with each set of dummy variables created
if 'Class' in chosenstats:
    
    if 'Conf' in chosenstats:
        print("found Conf and Class")
        # Create dummy variables for
        model_df = pd.get_dummies(model_df)
        model_df=model_df.drop(columns = ['Class_SR'])
        model_df=model_df.drop(columns = ['Conf_WAC'])
    else:
        print("found Class")
        # Create dummy variables for
        model_df = pd.get_dummies(model_df)
        model_df=model_df.drop(columns = ['Class_SR'])
else:
    if 'Conf' in chosenstats:
        print("found Conf")
        model_df = pd.get_dummies(model_df)
        model_df=model_df.drop(columns = ['Conf_WAC'])
    else:
        print("neither class or conference selected") 

#Create df of players that I want to predict who were drafted in 2020
pre2020draft = model_df[model_df['year']==2020]

#drop irrelevant columns of ID and year that I kept til now to verify I would know the subjects and be able to separate 
#out for year after creating dummy variable.  Technically if I wanted to keep year I should make it categorical
model_df.drop(columns = ['year','ID'], inplace = True)
pre2020draft.drop(columns = ['year','ID','FPTS'], inplace = True)

#Unsure if I want to train with data of players that didn't score any fantasy points, so may run below
if exclude:
    model_df = model_df[model_df['FPTS']>0]
else:
    pass

print(model_df['FPTS'].min())
#define x the inputs and y the actuals
X = model_df.drop(columns = ['FPTS'])
y = model_df['FPTS'].values.reshape(-1,1)
print(X.shape, y.shape,pre2020draft.shape)
# Xplay =play.drop(columns = ['FPTS'])
# yplay = play['FPTS'].values.reshape(-1,1)
# print(Xplay.shape,yplay.shape)
size = len(X.columns)
print(size)
# Split the data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 0.9,random_state=28)

#define the model
def build_model():
  model = keras.Sequential([
    layers.Dense(3*size, activation='relu', input_dim=size),
#     layers.Dense(3*size, activation='relu'),
#       layers.Dense(2*size, activation='relu'),
    layers.Dense(1)
  ])

  optimizer = tf.keras.optimizers.RMSprop(0.001)

  model.compile(loss='mse',
                optimizer=optimizer,
                metrics=['mae', 'mse'])
  return model

#build model for all data
model = build_model()
playmodel = build_model()

#fit training data to the model
model.fit(
    X_train,
    y_train,
    epochs=60,
    shuffle=True,
    verbose=0
)

#Test unscaled model with all players
loss, mae, mse = model.evaluate(
    X_test, y_test, verbose=2)
print(
    f"Normal Neural Network - Loss: {loss}, MSE: {mse}")

projected = model.predict(pre2020draft)

subjects2020['Projected Points']=projected
# subjects2020['playprojected points']=playprojectedpts

subjects2020.sort_values('Projected Points', ascending = False, inplace =True)
subjects2020.drop(columns = ['FirstNm','LastNm','ID'], inplace = True)
subjects2020.reset_index(inplace = True)

subjects2020['Rank']=subjects2020['index']+1

#reformat data for web display
web_display = subjects2020[['Rank','Player_x','Projected Points']]

web_display.rename(columns = {'Player_x': 'Player'}, inplace = True)


web_display
# #get data in a format to write to a file that can be read by webpage
data = web_display.to_dict('records')



neither class or conference selected
1.3
(169, 17) (169, 1) (36, 17)
17


C:\Users\walsh\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


1/1 - 0s - loss: 50081.7070 - mae: 168.2735 - mse: 50081.7070
Normal Neural Network - Loss: 50081.70703125, MSE: 50081.70703125


C:\Users\walsh\anaconda3\lib\site-packages\ipykernel_launcher.py:131: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\walsh\anaconda3\lib\site-packages\ipykernel_launcher.py:134: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\walsh\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
C:\Users\walsh\anaconda

In [210]:
# with open('static/js/outputfile.js', 'w') as writer:
#     writer.write("resultsdata=")
# #     for line in data:
# #         writer.write(line)
#     json.dump(data, writer)
# #     writer.write("]")

In [217]:

optdata = ['Conf','Class','Age', 'Wt', 'Forty', 'Vertical', 'BenchReps', 'BroadJump', 'ThreeCone', 'Shuttle',
'careerRec', 'careerRecYds', 'careerYPC', 'careerRecTds',
'careerRushAtt', 'careerRushYds', 'careerYPA', 'careerRushTds',
'careerPlays', 'careerTotalYds', 'careerYPT', 'careerTotalTds',
'dr_1games', 'dr_1Rec', 'dr_1RecYds',
'dr_1YPC', 'dr_1RecTds', 'dr_1RushAtt', 'dr_1RushYds', 'dr_1YPA',
'dr_1RushTds', 'dr_1Plays', 'dr_1TotalYds', 'dr_1YPT', 'dr_1TotalTds',
'dr_2games', 'dr_2Rec', 'dr_2RecYds', 'dr_2YPC', 'dr_2RecTds',
'dr_2RushAtt', 'dr_2RushYds', 'dr_2YPA', 'dr_2RushTds', 'dr_2Plays',
'dr_2TotalYds', 'dr_2YPT', 'dr_2TotalTds', 'dr_3games', 'dr_3Rec',
'dr_3RecYds', 'dr_3YPC', 'dr_3RecTds', 'dr_3RushAtt', 'dr_3RushYds',
'dr_3YPA', 'dr_3RushTds', 'dr_3Plays', 'dr_3TotalYds', 'dr_3YPT',
'dr_3TotalTds']


In [218]:
len(optdata)

61

In [250]:
web_display

,Rank,Player,Projected Points
0,2,Freddie Swain,305.272827
1,31,Quintez Cephus,278.387939
2,19,Isaiah Hodgins,269.905090
3,28,Gabriel Davis,263.963501
4,13,CeeDee Lamb,260.695648
5,33,Malcolm Perry,250.306076
6,6,Jalen Reagor,247.818802
7,22,Tee Higgins,243.960205
8,4,Henry Ruggs III,220.836624
9,17,Jauan Jennings,214.574097


In [209]:
data

[{'Rank': 1,
  'Player_x': 'Van Jefferson',
  'projected points': 708.7838745117188},
 {'Rank': 2,
  'Player_x': 'Brandon Aiyuk',
  'projected points': 656.5245361328125},
 {'Rank': 3,
  'Player_x': 'Tyler Johnson',
  'projected points': 355.78155517578125},
 {'Rank': 4,
  'Player_x': 'Jalen Reagor',
  'projected points': 352.3506164550781},
 {'Rank': 5,
  'Player_x': 'Laviska Shenault Jr.',
  'projected points': 342.65020751953125},
 {'Rank': 6,
  'Player_x': 'Darnell Mooney',
  'projected points': 298.54254150390625},
 {'Rank': 7,
  'Player_x': 'Donovan Peoples-Jones',
  'projected points': 289.5994873046875},
 {'Rank': 8,
  'Player_x': 'Justin Jefferson',
  'projected points': 268.51153564453125},
 {'Rank': 9,
  'Player_x': 'Antonio Gibson',
  'projected points': 263.41937255859375},
 {'Rank': 10,
  'Player_x': 'Dezmon Patmon',
  'projected points': 246.4557342529297},
 {'Rank': 11,
  'Player_x': 'Antonio Gandy-Golden',
  'projected points': 244.064453125},
 {'Rank': 12,
  'Player_x

In [51]:
# #read in the model csv
# model_df = pd.read_csv("modelPython.csv")

# predict2020 = pd.read_csv('draft2020.csv')
# #create a df of the subjects matched by ID
# subjects = model_df[['ID','Player_x','FirstNm','LastNm']]
# subjects2020 = predict2020[['ID','Player_x','FirstNm','LastNm']]



predict2020[['Wt', 'height','Age']] = predict2020[['Wt', 'height','Age']].astype(float)

# add required extras to the list of variable for the df
train_stats = chosenstats.copy()
train_stats.append('FPTS')

# list of 2020 players that the model is predicting
subjects2020 = predict2020[['ID','Player_x','FirstNm','LastNm']]

# create df's with only the variables chosen
traindf = model_df[train_stats]
pred2020 =predict2020[chosenstats]

#drop categorical columns that shouldn't be made to dummy variables


predict2020 = pd.get_dummies(predict2020)

#drop 1 columns associated with each set of dummy variables created
if 'Class' in chosenstats:
    print("found Class")
    model_df=model_df.drop(columns = ['Class_SR'])
    predict2020=predict2020.drop(columns = ['Class_SR'])
else:
    print("Class not selected")
if 'Conf' in chosenstats:
    print("found Conf")
    model_df=model_df.drop(columns = ['Class_SR'])
    predict2020=predict2020.drop(columns = ['Class_SR'])
else:
      pass    
    


#Uncomment belowt to view the columns of the model df
# for col in model_df.columns:
#     print(col)

#Eliminate players who didn't score a single fantasy point
# play = model_df[model_df['FPTS']!=0]
# play